In [1]:
# Importing the libraries
#14065234
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.dataframe as dd

In [2]:
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [3]:
# Importing the dataset
dataset = pd.read_csv("/scratch/ope4/MERGE/BUFFER_EXPORT/combined_dataset_BUFFER.csv")

# Drop the 'Unnamed: 0' column from the DataFrame
dataset = dataset.drop(columns=['Unnamed: 0'])

# Check the columns after dropping
print(dataset.columns)

#dataset = dd.read_csv("/scratch/ope4/MERGE/RF.csv")
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
print(X)
print(y)

Index(['LANDSAT_1', 'LANDSAT_2', 'LANDSAT_3', 'LANDSAT_4', 'LANDSAT_5',
       'LANDSAT_6', 'LANDSAT_7', 'LANDSAT_8', 'LANDSAT_9', 'prec', 'tmax',
       'tmin', 'status'],
      dtype='object')
[[ 3.32226654  3.50662709  4.16367833 ...  0.08843354  0.83283381
   1.30459184]
 [ 3.4005      3.58373476  4.27122115 ...  0.08843354  0.83283381
   1.30459184]
 [ 3.13058953  3.30050287  3.83548013 ...  0.08843354  0.83283381
   1.30459184]
 ...
 [-1.38825362 -1.21587871 -1.38409883 ...  1.51947837 -2.96539623
  -2.33836453]
 [-1.43071244 -1.27347081 -1.42713114 ...  1.51947837 -2.96539623
  -2.33836453]
 [-1.49853876 -1.36583453 -1.49699622 ...  1.51947837 -2.96539623
  -2.33836453]]
[0 0 0 ... 0 0 0]


In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[ 0.2531422   0.36860579  0.26347643 ... -0.24703722 -1.58422152
  -1.1240456 ]
 [-1.48347531 -1.41196983 -1.33330981 ...  1.06029553 -1.41157489
  -1.03730863]
 [-0.76599037 -0.65444447 -0.36091408 ...  0.68358762 -0.80731126
  -0.16993802]
 ...
 [ 0.39762626  0.5966089   0.66489936 ...  1.2163286  -0.97995789
  -1.21078279]
 [ 1.33936765  1.48443965  1.40162523 ...  0.47071418 -0.37569426
  -0.08320105]
 [-1.06538254 -1.12985425 -1.03211788 ...  0.59665493  0.48753972
  -0.34341219]]
[[-0.3394607  -0.33644672 -0.43390624 ... -1.55548452 -1.32525116
  -1.73120507]
 [ 0.30209329  0.48175501  0.6166087  ...  0.77832192  0.48753972
   0.69743258]
 [ 1.03409673  1.06585156  1.21833491 ... -0.29050346 -0.03040018
   0.09027332]
 ...
 [-0.63748228 -0.74738867 -0.43061956 ... -1.53653771  0.31489309
  -0.08320105]
 [ 0.38400262  0.15624498  0.63763581 ... -0.84442029 -1.23892825
  -0.60362332]
 [-2.11060539 -1.99203668 -1.96657876 ...  1.59415105  0.57386345
   2.34543681]]
[0 0 0 ... 0 0 1

In [5]:
#dataset.to_dask_array(lengths=True)

##X.compute_chunk_sizes()
#y.compute_chunk_sizes()


In [6]:
#from dask_ml.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, shuffle=True)


In [7]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)


y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




# Export the result to a text file
with open("accuracy_result.txt", "w") as file:
    file.write(f"Accuracy: {accuracy}\n")

Accuracy: 0.8859141791044776


In [8]:
# Predicting the Test set results
y_pred = rf.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))



# Export the result to a text file
with open("y_pred.txt", "w") as file:
    file.write(f"y_pred: {y_pred}\n")

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [9]:
# Export the first three decision trees from the forest

#for i in range(3):
#    tree = rf.estimators_[i]
 #   dot_data = export_graphviz(tree,
#                               feature_names=X_train.columns,  
#                               filled=True,  
#                               max_depth=2, 
#                               impurity=False, 
#                               proportion=True)
#    graph = graphviz.Source(dot_data)
#    display(graph)

In [10]:
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)






# Extract results
best_params = rand_search.best_params_
best_score = rand_search.best_score_

# Print to console
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Export results to a file
with open("random_search_results.txt", "w") as file:
    file.write("Randomized Search Results\n")
    file.write("=========================\n")
    file.write(f"Best Parameters: {best_params}\n")
    file.write(f"Best Cross-Validated Score: {best_score:.4f}\n")

Best Parameters: {'max_depth': 5, 'n_estimators': 309}
Best Score: 0.9096460814617927


In [ ]:
# Create a random forest classifier
rf = RandomForestClassifier()

# Fit the random search object to the data
rand_search.fit(X_train, y_train)


In [ ]:
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)


# Export the result to a text file
with open("best_rf.txt", "w") as file:
    file.write(f"best_rf: {best_rf}\n")

In [ ]:
# Generate predictions with the best model
y_pred = best_rf.predict(X_test)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
ConfusionMatrixDisplay(confusion_matrix=cm).plot();









# Export the confusion matrix to a text file
with open("confusion_matrix.txt", "w") as file:
    file.write("Confusion Matrix:\n")
    file.write("=================\n")
    file.write(f"{cm}\n")

# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(6, 6))
ConfusionMatrixDisplay(confusion_matrix=cm).plot(ax=ax)

# Save the confusion matrix plot as an image
plt.savefig("confusion_matrix_plot.png")
plt.show()

In [ ]:
# Select the first 12 columns for X
X = dataset.iloc[:, :-1].values  # First 12 columns

# Select the last column for y
y = dataset.iloc[:, -1].values  # Last column as the target

# Check the shape of X and y to ensure it's correct
print(X.shape)  # Should be (310761, 12)
print(y.shape)  # Should be (310761,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=335, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
accuracy_before = rf.score(X_test, y_test)
print(f'Accuracy before feature selection: {accuracy_before:.2f}')

In [ ]:
# If X is a numpy array, convert it back to a DataFrame
feature_names = ['LANDSAT_1', 'LANDSAT_2', 'LANDSAT_3', 'LANDSAT_4', 'LANDSAT_5', 'LANDSAT_6', 
                 'LANDSAT_7', 'LANDSAT_8', 'LANDSAT_9', 'prec', 'tmax', 'tmin']  # Manually specify feature names

X = pd.DataFrame(X, columns=feature_names)


In [ ]:
# If X has 13 columns, provide 13 column names
X = pd.DataFrame(X, columns=['LANDSAT_1', 'LANDSAT_2', 'LANDSAT_3', 'LANDSAT_4', 'LANDSAT_5', 'LANDSAT_6', 'LANDSAT_7', 'LANDSAT_8', 'LANDSAT_9', 'prec', 'tmax', 'tmin'])  # Adjust this list to include 13 names


In [ ]:
importances = rf.feature_importances_

# Create a list of feature names
feature_names = X.columns

# Create the DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Rank features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the results
print(feature_importance_df)


In [ ]:
# Assuming feature_names contains the correct column names
feature_names = list(X.columns)  # Retrieve column names from the original dataset

# Convert X_train and X_test to pandas DataFrames
X_train = pd.DataFrame(X_train, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

# Extract the top 5 features by importance
top_features = feature_importance_df['Feature'][0:5].values

# Select the top features
X_train_selected = X_train[top_features]  # Select top features by column names
X_test_selected = X_test[top_features]    # Same for the test set

# Display results
print("Selected Training Features:")
print(X_train_selected.head())

print("\nSelected Testing Features:")
print(X_test_selected.head())

In [ ]:
X_train_selected = X_train[top_features]

In [ ]:
X_test_selected = X_test[top_features]

In [ ]:
# Train the Random Forest model with selected features
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Evaluate the model
accuracy_after = rf_selected.score(X_test_selected, y_test)
print(f'Accuracy after feature selection: {accuracy_after:.2f}')

In [ ]:
print(X_train_selected)